用于对化合物进行分类


In [93]:
import pandas as pd
import numpy as np
import json
import re
import string
from tqdm import tqdm
from string import digits,punctuation
from gensim.parsing.preprocessing import remove_stopwords
import pymysql
import matplotlib.pyplot as plt 
import seaborn as sns

In [79]:
df_dic =pd.read_csv('data/Mapdic_Secondstep_1129.csv',index_col=0,keep_default_na=False,encoding = 'gbk')
df_dic.head()
dic_map_norm = {k:v for k,v in zip(df_dic.index,df_dic['Norm'])if v}
dic_map_category = {k:v for k,v in zip(df_dic.index,df_dic['Category'])if v}

In [ ]:
df = pd.read_csv('result_data/ph_temp_conc_ironsplit_results_20211129.csv',index_col = 0, keep_default_na = False)
data = df.copy()

In [ ]:
data['Norm_name'] = data['units_full_results'].map(
    lambda x:{dic_map_norm[k]:v for k,v in deal_with_nan(x).items() if k in dic_map_norm.keys() if x and k}
)
data['Norm_substance'] = data['units_full_results'].map(
    lambda x:[dic_map_norm[k] for k,_ in deal_with_nan(x).items() if k in dic_map_norm.keys() if x and k]
)
data['Category'] = data['units_full_results'].map(
    lambda x:[dic_map_category[k] for k,_ in eval(x).items() if k in dic_map_category.keys() if x and k]
)

In [ ]:
data[list(data.columns[:5])+list(data.columns[-3:])].to_csv('result_data/category_of_substance.csv')

In [94]:
data

,pdb_id,entity_id,seq,method,details,method_results,ph_results,temperature_results,units_full,units_full_results,iron_split_results,Norm_name,Category,Norm_substance
0,200l,1,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,,pH 6.7,[],['ph 6.7'],[],{},{},{},{},[],[]
1,101m,1,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",[],['ph 9.0'],[],"{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium,sulfate': ('3.0 m', '3.0', 'm', '3....","{'NH4SO4': ('3.0 m', '3.0', 'm', '3.0 m ammoni...","[沉淀剂, 缓冲试剂, 添加剂]","[NH4SO4, Tris, EDTA]"
2,102m,1,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",[],['ph 9.0'],[],"{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium,sulfate': ('3.0 m', '3.0', 'm', '3....","{'NH4SO4': ('3.0 m', '3.0', 'm', '3.0 m ammoni...","[沉淀剂, 缓冲试剂, 添加剂]","[NH4SO4, Tris, EDTA]"
3,103m,1,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",[],['ph 9.0'],[],"{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium,sulfate': ('3.0 m', '3.0', 'm', '3....","{'NH4SO4': ('3.0 m', '3.0', 'm', '3.0 m ammoni...","[沉淀剂, 缓冲试剂, 添加剂]","[NH4SO4, Tris, EDTA]"
4,105m,1,VLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRF...,,"3.0 M AMMONIUM SULFATE, UNBUFFERED, pH 9.0",[],['ph 9.0'],[],"{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium sulfate': ('3.0 m', '3.0', 'm', '3....","{'ammonium,sulfate': ('3.0 m', '3.0', 'm', '3....","{'NH4SO4': ('3.0 m', '3.0', 'm', '3.0 m ammoni...",[沉淀剂],[NH4SO4]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220132,3zzn,1,MKVGIVGSGMVGSATAYALALLGVAREVVLVDLDRKLAQAHAEDIL...,"VAPOR DIFFUSION, HANGING DROP",5-MUT TT-LDH WAS CRYSTALLIZED AT 10MG/ML BY TH...,"['vapor diffusion', 'hanging drop']",['ph 9'],[],{'mut ttldh crystallized 10mgml drop diffusion...,"{'peg6000': ('18', '18', '', '18-20% peg 6000'...","{'peg6000': ('18', '18', '', '18-20% peg 6000'...","{'PEG6000': ('18', '18', '', '18-20% peg 6000'...","[沉淀剂, 盐]","[PEG6000, LiCl]"
220133,2zzc,1,MAHHHHHHVDDDDKMNGPEDLPKSYDYDLIIIGGGSGGLAAAKEAA...,"VAPOR DIFFUSION, SITTING DROP","5% PEG 3350, 0.005M Magnesium Sulfate, 0.05M M...","['sitting drop', 'vapor diffusion']",['ph 6.0'],['temperature 298k'],"{'peg 3350': ('5%', '5', '%', '5% peg 3350'), ...","{'peg3350': ('5%', '5', '%', '5% peg 3350'), '...","{'peg3350': ('5%', '5', '%', '5% peg 3350'), '...","{'PEG3350': ('5%', '5', '%', '5% peg 3350'), '...","[沉淀剂, 盐, 缓冲试剂, 有机溶剂]","[PEG3350, MgSO4, MES, hexanediol]"
220134,5zzj,1,MDSSTATAMTAPFIDPTDHVNLKTDTDASENRRMGNYKPSIWNYDF...,"VAPOR DIFFUSION, SITTING DROP","Ammonium tartrate, glycerol","['sitting drop', 'vapor diffusion']",[],[],"{'ammonium tartrate': ('', '', '', 'ammonium t...","{'ammonium tartrate': ('', '', '', 'ammonium t...","{'ammonium,tartrate': ('', '', '', 'ammonium t...","{'ammonium tartrate': ('', '', '', 'ammonium t...","[缓冲试剂, 添加剂]","[ammonium tartrate, glycerol]"
220135,6zzr,1,MGGSHHHHHHENLYFQGATLKDQLIYNLLKEEQTPQNKITVVGVGA...,"VAPOR DIFFUSION, SITTING DROP","0.2M Na malonate, 0.1M Bis-Tris propane pH6.5,...","['sitting drop', 'vapor diffusion']",['ph6.5'],[],"{'na malonate': ('0.2m', '0.2', 'm', '0.2m na ...","{'sodium malonate': ('0.2m', '0.2', 'm', '0.2m...","{'sodium,malonate': ('0.2m', '0.2', 'm', '0.2m...","{'sodium malonate': ('0.2m', '0.2', 'm', '0.2m...","[缓冲试剂, 缓冲试剂, 沉淀剂]","[sodium malonate, Bis-Tris propane, PEG3]"
